In [31]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from itertools import product
from gensim.models import word2vec
import itertools
from tqdm import tqdm 

In [24]:
# Read recipe inputs
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [25]:
folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [26]:
store_cross = []
for ids in product(df['id'], repeat=2):
    store_cross.append(ids)

In [27]:
store_cross_df = pd.DataFrame(store_cross, columns=['id_x', 'id_y'])

In [28]:
store_cross_detail = store_cross_df.merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                          how='inner', 
                                          left_on='id_x', 
                                          right_on='id').drop(columns='id').merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                                                                  how='inner', 
                                                                                  left_on='id_y', 
                                                                                  right_on='id').drop(columns='id')

In [29]:
store_cross_detail.head()

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
1,ID-000002,ID-000001,Homemade Ramen 麦苗,3.97,731,"['記帳', 'ボード', 'ウェイティング', '深谷', '9時', '記名', 'ワン...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
2,ID-000003,ID-000001,麺尊 RAGE,3.96,723,"['金華', 'ハム', 'ロック', '穂先', '燻製', 'レモン', '豚バラ', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
3,ID-000004,ID-000001,宍道湖しじみ中華蕎麦 琥珀,3.95,296,"['シジミ', 'バラ', '焼き', '紫玉ねぎ', '吊し', '菅野', 'ワンタン'...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
4,ID-000005,ID-000001,らぁ麺や 嶋,3.95,177,"['記帳', '4点', '細切り', '記入', '炭火焼き', '昆布', '燻製', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."


In [30]:
# store_cross_detail = store_cross_detail[store_cross_detail['id_x'].isin(df['id'].loc[0:50])]
store_cross_detail = store_cross_detail.reset_index(drop=True).sort_values(['id_x'])

In [32]:
##ラーメン店xに対してラーメン店yの類似度を算出
#コサイン類似度を算出する関数を定義
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [0]:
#cossimだけの組み合わせ（同じワード同士の組みあわせがでてくるため）
#2次元を１次元にする　setが重複を削除てきなやつ。
# uniq_words = list(set(itertools.chain.from_iterable(df['texts_tfidf_sorted_top20'].values)))
word_list = []
arr = df['texts_tfidf_sorted_top20'].values
for a in arr:
    l = a.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    word_list.append(l)
uniq_words = list(set(itertools.chain.from_iterable(word_list)))    
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    # print(word1, word2)
    scores[(word1, word2)] =  cos_sim(ramen_model.wv[word1], ramen_model.wv[word2])

豚脂 豚脂
豚脂 iphone
豚脂 厳禁
豚脂 31分
豚脂 建物
豚脂 狭い
豚脂 とうがらし
豚脂 うま
豚脂 41
豚脂 お湯
豚脂 ジェル
豚脂 つき
豚脂 ソースカツ丼
豚脂 団体
豚脂 おば
豚脂 ベーコン
豚脂 兄さん
豚脂 メタル
豚脂 凄まじい
豚脂 香辛料
豚脂 ボイル
豚脂 煮玉
豚脂 テープ
豚脂 白濁
豚脂 冷た
豚脂 ガリマヨ
豚脂 注ぎ口
豚脂 スプラウト
豚脂 鶏ガラ
豚脂 掛け
豚脂 焦げ
豚脂 小ぶり
豚脂 ーー
豚脂 木製
豚脂 貝出
豚脂 三河屋
豚脂 土屋
豚脂 コート
豚脂 豚骨スープ
豚脂 ドトール
豚脂 スタンプカード
豚脂 オレンジ色
豚脂 事項
豚脂 薬膳
豚脂 佃煮
豚脂 豚バラ
豚脂 七輪
豚脂 全開
豚脂 親父
豚脂 配分
豚脂 蒲鉾
豚脂 出迎え
豚脂 付与
豚脂 ニボスープ
豚脂 上等
豚脂 焙煎
豚脂 タブレット
豚脂 21時
豚脂 物足りな
豚脂 塩っぱく
豚脂 お客
豚脂 快感
豚脂 鶏団子
豚脂 控え目
豚脂 肉味噌
豚脂 バイク
豚脂 集合
豚脂 ピカイチ
豚脂 ビニール
豚脂 霧島
豚脂 好感
豚脂 伊勢海老
豚脂 個性的
豚脂 アルデンテ
豚脂 お麩
豚脂 大蒜
豚脂 蕎麦屋
豚脂 岩のり
豚脂 鶏の胸肉
豚脂 幸い
豚脂 ワタ
豚脂 カップル
豚脂 白い
豚脂 ピンクペッパー
豚脂 コンビーフ
豚脂 ロールチャーシュー
豚脂 指南
豚脂 挨拶
豚脂 エグ
豚脂 カフェ
豚脂 豚肩ロース
豚脂 qrコード
豚脂 2階
豚脂 ウマイ
豚脂 黒っぽい
豚脂 セロリ
豚脂 スゲー
豚脂 アタマ
豚脂 ルール
豚脂 本節
豚脂 サイン
豚脂 口中
豚脂 シャバ
豚脂 フレーク
豚脂 細切れ
豚脂 苦み
豚脂 グラス
豚脂 辛味
豚脂 ヤミツキ
豚脂 サッパリ
豚脂 モヤシ
豚脂 広い
豚脂 ガニ
豚脂 天邪鬼
豚脂 最終的
豚脂 にくい
豚脂 西山
豚脂 ゼリー
豚脂 電話
豚脂 ドロ
豚脂 お通し
豚脂 ウェーブ
豚脂 薄暗く
豚脂 胡椒
豚脂 煮豚
豚脂 フレッシュ
豚脂 七福
豚脂 金華
豚脂 シャキ
豚脂 お餅
豚脂 エッジ
豚脂 旨かっ
豚脂 抽出
豚脂 イタリア産
豚脂 物腰
豚脂 陽気
豚脂 薄切り
豚脂 カーブ
豚脂 小料理
豚脂 太平
豚脂 エスプーマ
豚脂 酒井
豚脂 烏龍茶


iphone 苦み
iphone グラス
iphone 辛味
iphone ヤミツキ
iphone サッパリ
iphone モヤシ
iphone 広い
iphone ガニ
iphone 天邪鬼
iphone 最終的
iphone にくい
iphone 西山
iphone ゼリー
iphone 電話
iphone ドロ
iphone お通し
iphone ウェーブ
iphone 薄暗く
iphone 胡椒
iphone 煮豚
iphone フレッシュ
iphone 七福
iphone 金華
iphone シャキ
iphone お餅
iphone エッジ
iphone 旨かっ
iphone 抽出
iphone イタリア産
iphone 物腰
iphone 陽気
iphone 薄切り
iphone カーブ
iphone 小料理
iphone 太平
iphone エスプーマ
iphone 酒井
iphone 烏龍茶
iphone air
iphone エシャロット
iphone 紫蘇
iphone 内臓
iphone 鮮明
iphone 派手
iphone パウダー
iphone エビ
iphone 濃いめ
iphone 割り
iphone 衝立
iphone 防止
iphone 丸く
iphone シンプル
iphone 両立
iphone 生地
iphone 粉末
iphone めんま
iphone 切れ端
iphone 外人
iphone 端麗
iphone アラ
iphone 想定内
iphone 紅白
iphone ロッド
iphone へん
iphone 包丁
iphone 向き
iphone ゲット
iphone 能書き
iphone 十二分
iphone スタイル
iphone 固い
iphone タラ
iphone 猪又
iphone 梅干し
iphone 左右
iphone ペッパー
iphone 薫香
iphone 臭み
iphone 西山製麺
iphone 拍子抜け
iphone 硬い
iphone 塩辛かっ
iphone 38
iphone すり身
iphone 小声
iphone 緻密
iphone トゥルン
iphone ドレッシング
iphone 熱心
iphone 水平
iphone 塩っぱい
iphone 青唐辛子
iphone 気配り


iphone お米
iphone ツル
iphone ニンニク
iphone 魚の
iphone 半券
iphone 玉ネギ
iphone 平太
iphone 可愛く
iphone 女性
iphone ドボン
iphone サバ
iphone ローテーション
iphone おしゃれ
iphone 作り
iphone 低温
iphone リーフ
iphone 油膜
iphone 3種
iphone 西京
iphone お肉
iphone 柑橘系
iphone コッテリ
iphone 採用
iphone 縮れ麺
iphone クラゲ
iphone メリハリ
iphone 長ネギ
iphone 柚子胡椒
iphone アサリ
iphone 塩加減
iphone 石臼挽き
iphone とも
iphone 三分
iphone 華やかさ
iphone ムッチリ
iphone 黄色
iphone マッシュルーム
iphone ol
iphone やわらかく
iphone ありか
iphone でかい
iphone 重厚
iphone イクラ
iphone 野菜
iphone ブラックペッパー
iphone ガラス
iphone フライドガーリック
iphone ゆず
iphone パプリカ
iphone 黒ラベル
iphone 背番号
iphone ヘルシー
iphone 動揺
iphone 1時間前
iphone 手渡し
iphone 繁盛店
iphone 赤褐色
iphone タマリンド
iphone 香味
iphone 応え
iphone ハイブリッド
iphone コース料理
iphone 受付
iphone 鉄板
iphone 明る
iphone シート
iphone パフォーマンス
iphone 炭水化物
iphone 大山どり
iphone 3割
iphone ほうとう
iphone 再来
iphone 14
iphone レモン
iphone 九条葱
iphone コロナ対応
iphone 待合
iphone ローストポーク
iphone 調理方法
iphone 名簿
iphone ハード
iphone 無化調
iphone なめこ
iphone シャクシャク
iphone 悪天候
iphone グッズ
iphone デロデロ
iphone 甘辛い
iphone

厳禁 マイルド
厳禁 格闘
厳禁 タケノコ
厳禁 キッチン
厳禁 rdb
厳禁 モニター
厳禁 探索
厳禁 隔離
厳禁 待ち人
厳禁 細長
厳禁 レストラン
厳禁 20
厳禁 トロ
厳禁 おかしく
厳禁 乳白色
厳禁 青ねぎ
厳禁 喧騒
厳禁 腹ペコ
厳禁 がい
厳禁 つる
厳禁 オーション
厳禁 しゃぶしゃぶ
厳禁 カラメ
厳禁 豚骨
厳禁 一つ一つ
厳禁 撮影
厳禁 裏口
厳禁 半熟卵
厳禁 ズシリ
厳禁 ほうれん草
厳禁 オン
厳禁 岩中
厳禁 ニボニボ
厳禁 オーブン
厳禁 山水
厳禁 クシ
厳禁 隙間
厳禁 エスニック
厳禁 モミジ
厳禁 カイワレ大根
厳禁 ワンポイント
厳禁 葛西
厳禁 部分
厳禁 硬く
厳禁 危ない
厳禁 レア
厳禁 だし
厳禁 不規則
厳禁 ぷり
厳禁 クラシカル
厳禁 14分
厳禁 クミン
厳禁 小海老
厳禁 大鍋
厳禁 つるし
厳禁 苦味
厳禁 コレ
厳禁 ワシワシ
厳禁 ソープ
厳禁 液体
厳禁 暗く
厳禁 背後
厳禁 角切
厳禁 リフトアップ
厳禁 輪切り
厳禁 ランチョンマット
厳禁 2時間半
厳禁 北海道
厳禁 整理券
厳禁 長方形
厳禁 間引き
厳禁 濃密
厳禁 記入
厳禁 出して
厳禁 北海道産
厳禁 みょうが
厳禁 トンコツ
厳禁 切れ味
厳禁 美人
厳禁 対極
厳禁 厳選
厳禁 柚子
厳禁 ズズズ
厳禁 オブジェ
厳禁 煮込み
厳禁 どろどろ
厳禁 柚子の皮
厳禁 内観
厳禁 作戦
厳禁 10人
厳禁 女子
厳禁 ミスマッチ
厳禁 真っ白
厳禁 クタ
厳禁 バーナー
厳禁 肉汁
厳禁 すき
厳禁 貝割れ
厳禁 居抜き
厳禁 メール
厳禁 にこやか
厳禁 コーヒーサイフォン
厳禁 ゆで卵
厳禁 ワイルド
厳禁 スイカ
厳禁 生臭
厳禁 仕様
厳禁 配慮
厳禁 そうめん
厳禁 お札
厳禁 お洒落
厳禁 干し海老
厳禁 片口鰯
厳禁 キクラゲ
厳禁 プリッ
厳禁 丸大
厳禁 当日
厳禁 組み合わせ
厳禁 要請
厳禁 浅草開化楼
厳禁 深く
厳禁 ナチュラル
厳禁 スモーク
厳禁 たまねぎ
厳禁 ペースト状
厳禁 ウルメイワシ
厳禁 丸太
厳禁 煮汁
厳禁 人前
厳禁 オイリー
厳禁 9番
厳禁 挽き肉
厳禁 パツッ
厳禁 づる
厳禁 叉焼
厳禁 単体
厳禁 塩辛い
厳禁 スピーディー
厳禁 間際
厳禁 狭かっ
厳禁 ぼ

31分 豚骨スープ
31分 ドトール
31分 スタンプカード
31分 オレンジ色
31分 事項
31分 薬膳
31分 佃煮
31分 豚バラ
31分 七輪
31分 全開
31分 親父
31分 配分
31分 蒲鉾
31分 出迎え
31分 付与
31分 ニボスープ
31分 上等
31分 焙煎
31分 タブレット
31分 21時
31分 物足りな
31分 塩っぱく
31分 お客
31分 快感
31分 鶏団子
31分 控え目
31分 肉味噌
31分 バイク
31分 集合
31分 ピカイチ
31分 ビニール
31分 霧島
31分 好感
31分 伊勢海老
31分 個性的
31分 アルデンテ
31分 お麩
31分 大蒜
31分 蕎麦屋
31分 岩のり
31分 鶏の胸肉
31分 幸い
31分 ワタ
31分 カップル
31分 白い
31分 ピンクペッパー
31分 コンビーフ
31分 ロールチャーシュー
31分 指南
31分 挨拶
31分 エグ
31分 カフェ
31分 豚肩ロース
31分 qrコード
31分 2階
31分 ウマイ
31分 黒っぽい
31分 セロリ
31分 スゲー
31分 アタマ
31分 ルール
31分 本節
31分 サイン
31分 口中
31分 シャバ
31分 フレーク
31分 細切れ
31分 苦み
31分 グラス
31分 辛味
31分 ヤミツキ
31分 サッパリ
31分 モヤシ
31分 広い
31分 ガニ
31分 天邪鬼
31分 最終的
31分 にくい
31分 西山
31分 ゼリー
31分 電話
31分 ドロ
31分 お通し
31分 ウェーブ
31分 薄暗く
31分 胡椒
31分 煮豚
31分 フレッシュ
31分 七福
31分 金華
31分 シャキ
31分 お餅
31分 エッジ
31分 旨かっ
31分 抽出
31分 イタリア産
31分 物腰
31分 陽気
31分 薄切り
31分 カーブ
31分 小料理
31分 太平
31分 エスプーマ
31分 酒井
31分 烏龍茶
31分 air
31分 エシャロット
31分 紫蘇
31分 内臓
31分 鮮明
31分 派手
31分 パウダー
31分 エビ
31分 濃いめ
31分 割り
31分 衝立
31分 防止
31分 丸く
31分 シンプル
31分 両立
31分 生地
31分 粉末
31分 めんま
31分 切れ端
31分 外人
31分

...

出迎え 微妙
出迎え ジャンキー
出迎え スムーズ
出迎え 馴染み
出迎え 角切り
出迎え 白と黒
出迎え 昆布だし
出迎え ほろ
出迎え スパイシー
出迎え ノート
出迎え 寡黙
出迎え トリ
出迎え レモン酢
出迎え 極太
出迎え サラ系
出迎え モモ肉
出迎え 地味
出迎え レアチャーシュー
出迎え トロリ
出迎え 晴れ
出迎え 卵の黄身
出迎え やさし
出迎え しょっぱ
出迎え 待機
出迎え アラカルト
出迎え 炭火
出迎え 鶏皮
出迎え 大粒
出迎え 調味料
出迎え 無添加
出迎え っぱなし
出迎え 中力粉
出迎え 刻み葱
出迎え 瑞々しい
出迎え 天然
出迎え お仕事
出迎え 丸見え
出迎え 薬味
出迎え ザル
出迎え シナジ
出迎え 味わい深く
出迎え えのき
出迎え 背中合わせ
出迎え 窮屈
出迎え 支那竹
出迎え 大型
出迎え コンソメスープ
出迎え カニカマ
出迎え 先払い
出迎え 変哲
出迎え 隣席
出迎え きれい
出迎え ストップ
出迎え バゲット
出迎え 紫玉ねぎ
出迎え 真空
出迎え お姉さん
出迎え みじん
出迎え 温度
出迎え 熱燗
出迎え ディスタンス
出迎え 魚介
出迎え 好き嫌い
出迎え サーブ
出迎え 西京味噌
出迎え 胡麻油
出迎え ドライトマト
出迎え エキス
出迎え 人参
出迎え レンコン
出迎え 半熟玉子
出迎え ガッツリ
出迎え ふり
出迎え 鶏肉
出迎え のど越し
出迎え 持参
出迎え ソーキ
出迎え 千円札
出迎え 花椒
出迎え 男女
出迎え 受けて
出迎え 茎わかめ
出迎え epark
出迎え 豚ロース
出迎え 塩豚
出迎え 予報
出迎え チケット
出迎え あなた
出迎え 硬さ
出迎え 青森シャモロック
出迎え 気候
出迎え 豚ばら
出迎え ガラ
出迎え スナック
出迎え セメント
出迎え カク
出迎え 1120円
出迎え 柑橘
出迎え ゴワ
出迎え 間近
出迎え お茶
出迎え 比べ
出迎え 豆腐
出迎え ウェイト
出迎え テーブル席
出迎え ホット
出迎え ミソ
出迎え クリーム
付与 豚脂
付与 iphone
付与 厳禁
付与 31分
付与 建物
付与 狭い
付与 とうがらし
付与 うま
付与 41
付与 お湯
付与 ジェル
付与 つき
付与 ソースカツ丼
付与 団体
付与 おば
付与 ベーコン
付与 兄さ

付与 分厚く
付与 貝柱
付与 岩海苔
付与 サラ
付与 白胡麻
付与 煮卵
付与 ブロス
付与 ハワイ
付与 2本
付与 仄か
付与 ゚と
付与 存分
付与 外側
付与 白め
付与 サイン色紙
付与 ハム
付与 レアチャー
付与 ダシ
付与 アーリーレッド
付与 玄米
付与 紅ショウガ
付与 栄養
付与 注意書き
付与 既存
付与 刻みニンニク
付与 27分
付与 淡い
付与 ひとり
付与 細い
付与 ドゥエ
付与 25分
付与 ハンド
付与 ひと
付与 イキイキ
付与 トマトソース
付与 宮崎県
付与 玉葱
付与 ポット
付与 シロップ
付与 真昆布
付与 6枚
付与 鮮度
付与 パンパン
付与 瞬間
付与 入れ替え
付与 ベビーリーフ
付与 カツオ
付与 会津
付与 ピーナッツ
付与 line
付与 小さ
付与 最後列
付与 レタス
付与 でっかい
付与 アジ
付与 ニンジン
付与 洋風
付与 触り
付与 初め
付与 設計
付与 トレー
付与 煮干し
付与 上手い
付与 トイレ
付与 明るい
付与 ホロリ
付与 混濁
付与 嶋崎
付与 味つけ
付与 柔らかかっ
付与 赤い
付与 蓮根
付与 美し
付与 ぽい
付与 30人
付与 内外
付与 切り
付与 吊し
付与 段差
付与 ルッコラ
付与 セルフサービス
付与 刺激
付与 ち中
付与 うまし
付与 美しき
付与 紅生姜
付与 すき焼き
付与 湯上
付与 強面
付与 2種
付与 茶碗
付与 いか
付与 一番乗り
付与 20人
付与 2つ
付与 ハーブ
付与 食品
付与 晩酌
付与 真冬
付与 鮮魚
付与 水出
付与 オリジナル
付与 ペースト
付与 マナー
付与 糸唐辛子
付与 ダブル
付与 上澄み
付与 ピロピロ
付与 いちじく
付与 素地
付与 依頼
付与 良し
付与 粒胡椒
付与 ものすごく
付与 帆立
付与 鶏モモ肉
付与 正体
付与 2nd
付与 おかみ
付与 焙じ茶
付与 仕切り
付与 パリ
付与 桜台
付与 調整
付与 謙虚
付与 鶏胸肉
付与 黄金
付与 ウーロン茶
付与 コール
付与 対策
付与 お父さん
付与 レアチャーシュ
付与 ミョウガ
付与 鶏ハム
付与 緩和
付与 千寿
付与 ストーブ
付与 やすかっ
付与 tシャツ
付与 塩分
付与 ミリ
付与 まんま
付与 粘性
付与 わし
付与

ニボスープ 一陣
ニボスープ 青梗菜
ニボスープ 大量
ニボスープ 短め
ニボスープ 掃除
ニボスープ 巻き
ニボスープ 若林
ニボスープ 進み
ニボスープ 前述
ニボスープ 重たい
ニボスープ 今一つ
ニボスープ 豪快
ニボスープ ロース
ニボスープ モモ
ニボスープ 湯切り
ニボスープ 胚芽
ニボスープ 滋味
ニボスープ 小さく
ニボスープ ビニールシート
ニボスープ ポタージュ
ニボスープ サワークリーム
ニボスープ 静か
ニボスープ 蘊蓄
ニボスープ カオス
ニボスープ とろみ
ニボスープ 4人
ニボスープ 新鮮
ニボスープ おしぼり
ニボスープ お醤油
ニボスープ 17分
ニボスープ 女性客
ニボスープ ざく
ニボスープ 素早い
ニボスープ ちゃんぽん
ニボスープ 鶏手羽先
ニボスープ 間接照明
ニボスープ お盆
ニボスープ 本館
ニボスープ オニオン
ニボスープ 甲殻類
ニボスープ 歯切れ
ニボスープ 漆黒
ニボスープ 天下一品
ニボスープ 来店客
ニボスープ 軽快
ニボスープ 揚げ
ニボスープ ポルチーニデュクセル
ニボスープ 全面
ニボスープ ゲスト
ニボスープ 乳化
ニボスープ 束の間
ニボスープ かたく
ニボスープ ポリ
ニボスープ 反射
ニボスープ 及第点
ニボスープ ワンタン
ニボスープ オシャレ
ニボスープ 万能ネギ
ニボスープ 駐輪場
ニボスープ 細切り
ニボスープ 鶏ムネ肉
ニボスープ ヒバジ
ニボスープ 交代
ニボスープ 1本
ニボスープ 熟成
ニボスープ 脇役
ニボスープ 七味
ニボスープ 20時
ニボスープ 推測
ニボスープ ソース
ニボスープ 覆面
ニボスープ 鶏がら
ニボスープ ボトル
ニボスープ スーフ
ニボスープ 辛み
ニボスープ フォルム
ニボスープ たけのこ
ニボスープ ジュレ
ニボスープ さりげ
ニボスープ まぜまぜ
ニボスープ 1時間
ニボスープ ミンチ
ニボスープ ブロック
ニボスープ インカベリーソース
ニボスープ アワビ
ニボスープ 親子
ニボスープ 食べ方
ニボスープ 素朴
ニボスープ 筆者
ニボスープ 病みつき
ニボスープ 茨城県
ニボスープ カラスミ
ニボスープ ゴマ
ニボスープ 三密
ニボスープ シャモ
ニボスープ 万能ねぎ
ニボスープ あん肝
ニボスープ 濃ゆい
ニボスープ ぷりっぷり
ニボスープ 

上等 お気
上等 白葱
上等 愛想
上等 えぐみ
上等 濾過
上等 準備
上等 宣告
上等 ホスピタリティ
上等 共用
上等 サラミ
上等 硬かっ
上等 複数
上等 4つ
上等 材木
上等 方式
上等 丸鶏
上等 爽やか
上等 午前
上等 重さ
上等 芽菜
上等 細かく
上等 柔ら
上等 シジミ
上等 ワサビ
上等 シラス
上等 銀鱈
上等 余計
上等 色彩
上等 椎茸
上等 薫り
上等 物体
上等 お米
上等 ツル
上等 ニンニク
上等 魚の
上等 半券
上等 玉ネギ
上等 平太
上等 可愛く
上等 女性
上等 ドボン
上等 サバ
上等 ローテーション
上等 おしゃれ
上等 作り
上等 低温
上等 リーフ
上等 油膜
上等 3種
上等 西京
上等 お肉
上等 柑橘系
上等 コッテリ
上等 採用
上等 縮れ麺
上等 クラゲ
上等 メリハリ
上等 長ネギ
上等 柚子胡椒
上等 アサリ
上等 塩加減
上等 石臼挽き
上等 とも
上等 三分
上等 華やかさ
上等 ムッチリ
上等 黄色
上等 マッシュルーム
上等 ol
上等 やわらかく
上等 ありか
上等 でかい
上等 重厚
上等 イクラ
上等 野菜
上等 ブラックペッパー
上等 ガラス
上等 フライドガーリック
上等 ゆず
上等 パプリカ
上等 黒ラベル
上等 背番号
上等 ヘルシー
上等 動揺
上等 1時間前
上等 手渡し
上等 繁盛店
上等 赤褐色
上等 タマリンド
上等 香味
上等 応え
上等 ハイブリッド
上等 コース料理
上等 受付
上等 鉄板
上等 明る
上等 シート
上等 パフォーマンス
上等 炭水化物
上等 大山どり
上等 3割
上等 ほうとう
上等 再来
上等 14
上等 レモン
上等 九条葱
上等 コロナ対応
上等 待合
上等 ローストポーク
上等 調理方法
上等 名簿
上等 ハード
上等 無化調
上等 なめこ
上等 シャクシャク
上等 悪天候
上等 グッズ
上等 デロデロ
上等 甘辛い
上等 固定
上等 アツアツ
上等 両替
上等 48
上等 通知
上等 雪塩
上等 アイテム
上等 チョイ
上等 きしめん
上等 タッチパネル
上等 古民家
上等 吸い物
上等 90分
上等 開化
上等 壁際
上等 ネバネバ
上等 香ばしい
上等 茹で卵
上等 細か
上等 ムネ肉
上等 物足りなく
上等 麦茶
上等 暑

焙煎 ムース
焙煎 アニマルオフ
焙煎 足らず
焙煎 fz
焙煎 タピオカ粉
焙煎 ワンオペ
焙煎 脂っこく
焙煎 コーン
焙煎 ソテー
焙煎 特注
焙煎 段違い
焙煎 ウエイティングスペース
焙煎 学習
焙煎 退避
焙煎 清涼感
焙煎 色濃く
焙煎 刻み
焙煎 生クリーム
焙煎 テーブル
焙煎 通路
焙煎 洋楽
焙煎 大葉
焙煎 黒い
焙煎 思いのほか
焙煎 寸胴
焙煎 エグ味
焙煎 赤玉
焙煎 シャーベット
焙煎 ビター
焙煎 投入
焙煎 8点
焙煎 下品
焙煎 不揃い
焙煎 お刺身
焙煎 つけ出
焙煎 メグジ
焙煎 退店
焙煎 ベンチ
焙煎 クリーミー
焙煎 今時
焙煎 胸肉
焙煎 相応
焙煎 サラダ
焙煎 適量
焙煎 何気に
焙煎 海藻
焙煎 ビオレ
焙煎 羅臼
焙煎 多め
焙煎 ぬるい
焙煎 大陸
焙煎 アゴ
焙煎 コンソメ
焙煎 順番
焙煎 空き
焙煎 休止
焙煎 紅色
焙煎 カード
焙煎 スタイリッシュ
焙煎 コントロール
焙煎 飛沫
焙煎 辛さ
焙煎 生揚げ
焙煎 色白
焙煎 親切
焙煎 お酢
焙煎 辛く
焙煎 太い
焙煎 浸透
焙煎 長芋
焙煎 少な目
焙煎 便利
焙煎 乾物
焙煎 8番
焙煎 松村
焙煎 攻撃
焙煎 黄金色
焙煎 乱切り
焙煎 やわ
焙煎 コーンポタージュ
焙煎 ディップ
焙煎 日傘
焙煎 口頭
焙煎 マヨネーズ
焙煎 暖かい
焙煎 プチ
焙煎 会話
焙煎 早い
焙煎 高級感
焙煎 バツ
焙煎 商売
焙煎 ヒンヤリンコ
焙煎 水中
焙煎 挽肉
焙煎 ごま油
焙煎 山椒
焙煎 17時
焙煎 ニンニク醤油
焙煎 40人
焙煎 カイワレ
焙煎 のどごし
焙煎 レバーペースト
焙煎 ロール
焙煎 耐性
焙煎 鍋二郎
焙煎 緊張感
焙煎 携帯
焙煎 バルサミコ
焙煎 しなやか
焙煎 課長
焙煎 テーブルセット
焙煎 風貌
焙煎 腰つき
焙煎 ハマグリ
焙煎 一見さん
焙煎 2時間
焙煎 モツ
焙煎 ww
焙煎 分厚い
焙煎 キン
焙煎 満開
焙煎 9点
焙煎 りな
焙煎 ピーク
焙煎 おじ
焙煎 上手
焙煎 正統派
焙煎 入場
焙煎 厚い
焙煎 白菜
焙煎 牛肉
焙煎 うまみ
焙煎 一抹
焙煎 座敷
焙煎 チャッチャ
焙煎 開始
焙煎 デュラムセモリナ
焙煎 ムネ
焙煎 バリカタ
焙煎 味見
焙煎 極めて
焙煎 クタクタ
焙煎 つみれ
焙煎

In [50]:
df['texts_tfidf_sorted_top20'].values[0].replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [52]:
word_list = []
arr = df['texts_tfidf_sorted_top20'].values
for a in arr:
    l = a.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    word_list.append(l)

In [56]:
list(set(itertools.chain.from_iterable(word_list)))

['豚脂',
 'iphone',
 '厳禁',
 '31分',
 '建物',
 '狭い',
 'とうがらし',
 'うま',
 '41',
 'お湯',
 'ジェル',
 'つき',
 'ソースカツ丼',
 '団体',
 'おば',
 'ベーコン',
 '兄さん',
 'メタル',
 '凄まじい',
 '香辛料',
 'ボイル',
 '煮玉',
 'テープ',
 '白濁',
 '冷た',
 'ガリマヨ',
 '注ぎ口',
 'スプラウト',
 '鶏ガラ',
 '掛け',
 '焦げ',
 '小ぶり',
 'ーー',
 '木製',
 '貝出',
 '三河屋',
 '土屋',
 'コート',
 '豚骨スープ',
 'ドトール',
 'スタンプカード',
 'オレンジ色',
 '事項',
 '薬膳',
 '佃煮',
 '豚バラ',
 '七輪',
 '全開',
 '親父',
 '配分',
 '蒲鉾',
 '出迎え',
 '付与',
 'ニボスープ',
 '上等',
 '焙煎',
 'タブレット',
 '21時',
 '物足りな',
 '塩っぱく',
 'お客',
 '快感',
 '鶏団子',
 '控え目',
 '肉味噌',
 'バイク',
 '集合',
 'ピカイチ',
 'ビニール',
 '霧島',
 '好感',
 '伊勢海老',
 '個性的',
 'アルデンテ',
 'お麩',
 '大蒜',
 '蕎麦屋',
 '岩のり',
 '鶏の胸肉',
 '幸い',
 'ワタ',
 'カップル',
 '白い',
 'ピンクペッパー',
 'コンビーフ',
 'ロールチャーシュー',
 '指南',
 '挨拶',
 'エグ',
 'カフェ',
 '豚肩ロース',
 'qrコード',
 '2階',
 'ウマイ',
 '黒っぽい',
 'セロリ',
 'スゲー',
 'アタマ',
 'ルール',
 '本節',
 'サイン',
 '口中',
 'シャバ',
 'フレーク',
 '細切れ',
 '苦み',
 'グラス',
 '辛味',
 'ヤミツキ',
 'サッパリ',
 'モヤシ',
 '広い',
 'ガニ',
 '天邪鬼',
 '最終的',
 'にくい',
 '西山',
 'ゼリー',
 '電話',
 'ドロ',
 'お通し',
 'ウェーブ',
 '薄暗く',
 '胡椒',
 '煮豚',

In [16]:
avg_avg_scores = []
for i in tqdm(range(len(store_cross_detail['texts_tfidf_sorted_top20_x']))):
    avg_scores = []
    for j in range(len(store_cross_detail['texts_tfidf_sorted_top20_x'][i])):
        word_cross_scores = []
        word_a = store_cross_detail['texts_tfidf_sorted_top20_x'][i][j]
        for k in range(len(store_cross_detail['texts_tfidf_sorted_top20_y'][i])):
            word_b = store_cross_detail['texts_tfidf_sorted_top20_y'][i][k]
            score = scores[(word_a, word_b)]#単語間のスコアを出す。
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))#20個の単語間スコアの平均値
    avg_avg_scores.append(np.mean(avg_scores))#20個の単語間スコアの平均値の平均値
store_cross_detail.insert(6, 'avg_cos_sim_rate', avg_avg_scores)  
# 「二郎」と類似度が高いラーメン屋を高い順に表示
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
df_sim_x = store_cross_detail[store_cross_detail['store_name_x'].str.contains('二郎')]
df_sim_x.reset_index(drop=True)

def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result
b = df_sim_x['avg_cos_sim_rate']
c = min_max(b.values)
df_sim_x.insert(7, '正規化', c)
df_sim_x

NameError: name 'store_df' is not defined

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

similarity_scores_df = reviews_TF_IDF_df # For this sample code, simply copy input to output


# Write recipe outputs
similarity_scores = dataiku.Dataset("similarity_scores")
similarity_scores.write_with_schema(similarity_scores_df)